# To Vaccinate or Not to Vaccinate - Solution Notebook
## Loading data and libraries and preparing data

In [84]:
# loading libraries
library(tm)
library(SnowballC)
# install.packages('caTools')
library(caTools)
# install.packages('randomForest')
library(randomForest)
library(tidyverse)
library(caret)
#install.packages('xgboost')
library(xgboost)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ readr   1.3.1
✔ tibble  2.1.3     ✔ purrr   0.3.3
✔ tidyr   0.8.3     ✔ dplyr   0.8.1
✔ ggplot2 3.2.0     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ ggplot2::annotate() masks NLP::annotate()
✖ dplyr::arrange()    masks plyr::arrange()
✖ dplyr::combine()    masks randomForest::combine()
✖ purrr::compact()    masks plyr::compact()
✖ dplyr::count()      masks plyr::count()
✖ dplyr::failwith()   masks plyr::failwith()
✖ dplyr::filter()     masks stats::filter()
✖ dplyr::id()         masks plyr::id()
✖ dplyr::lag()        masks stats::lag()
✖ ggplot2::margin()   masks randomForest::margin()
✖ dplyr::mutate()     masks plyr::mutate()
✖ dplyr::rename()     masks plyr::rename()
✖ dplyr::summarise()  masks plyr::summarise()
✖ dplyr::summarize()  masks plyr::summarize()


In [75]:
# loading data
dataset_original = read.csv('Train.csv')
test = read.csv('Test (1).csv')
ss = read.csv('SampleSubmission.csv')

In [52]:
head(dataset_original)

tweet_id,safe_text,label,agreement
<fct>,<fct>,<dbl>,<dbl>
CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0,1.0000000
E3303EME,I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day,1,1.0000000
M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD",-1,1.0000000
1DR6ROZ4,"I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",-1,1.0000000
J77ENIIE,Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>,0,1.0000000
OVNPOAUX,<user> a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?,1,0.6666667


In [76]:
anyNA(dataset_original)

[1] TRUE

In [77]:
# Let us look at the rows with missing entries
unique (unlist (lapply (dataset_original, function (x) which (is.na (x)))))

[1] 4799 4800

In [78]:
dataset_original[c(4799 ,4800), ]

,tweet_id,safe_text,label,agreement
,<fct>,<fct>,<dbl>,<dbl>
4799,RQMQ0L2A,#lawandorderSVU,NA,NA
4800,I cannot believe in this day and age some parents could be so oblivious to reality as to not #vaccinate their child.,1,0.6666667,NA


In [79]:
dataset_original$agreement = NULL # we dont this column anyway for the nlp
#then drop missing rows 
dataset_original = na.exclude(dataset_original)

In [80]:
head(dataset_original)
dim(dataset_original)

tweet_id,safe_text,label
<fct>,<fct>,<dbl>
CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>,0
E3303EME,I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day,1
M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD",-1
1DR6ROZ4,"I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",-1
J77ENIIE,Thanks to <user> Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr… <url>,0
OVNPOAUX,<user> a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?,1


[1] 10000     3

In [ ]:
# Let us look at the rows with missing entries
anyNA(test)
unique (unlist (lapply (test, function (x) which (is.na (x)))))

In [ ]:
test = = na.exclude(test)

In [102]:
# Merger train and test data for ease of processing and prediction
# create new column separator in both datasets and populate 0 for train data and 1 for test data as follows
dataset_original$sep = 0
test$sep = 1


In [103]:
library(plyr)
new_dataset = rbind.fill(dataset_original, test)
new_dataset[sample(nrow(new_dataset), 10), ]

,tweet_id,safe_text,label,sep
,<fct>,<fct>,<dbl>,<dbl>
2145,CRS284RM,People getting vaccinated thinking they safe are just getting infected with Ebola..,-1,0
7138,Y93LLKPK,An extremely well presented video on the fallacious idea that vaccines can cause autism. RT: “<user> Watch: <url>,1,0
10884,61ZVUAQM,AM-News : How babies may help researchers build a better HIV vaccine: Most HIV vaccine research has focused on… <url>,NA,1
13329,N0Z2FSWP,"Calling All Up and Comin Artist #MMR &amp; #PDGFX brings To You ""Young &amp; Ready"" Concert Series Thursday… <url>",NA,1
5157,B10NC5CC,<user> #GOP STOP blaming #Immigrants first #Ebola now #measles Which diseases?brought their ancestors? #AINF <url>,0,0
7722,IFBGV42A,#blackbox very unlikely cause of autism. The immunization no longer contain mercury,1,0
8032,TZ1QG2YI,"So inspiring RT <user> Your weekly cynicism vaccine: ""Pakistani girls defy Taliban school bombings"" <url>",0,0
6304,Z345DGAA,<user> <user> This vaccine is called Gardicla vaccine &amp; is given 2 girls ages8-26 in USA. Gates was not trying 2 harm.Vaccine has,1,0
5631,ZO6LPGBN,"“<user> Thanks Jenny McCarthy! <url> this is horrible, and terrifying for new mothers, please vaccinate your children!",1,0


In [104]:
# removing user tags, replacing "amp" with "and" and removing url tags
library(stringr)
new_dataset$safe_text = str_replace(new_dataset$safe_text, "amp", "and")
new_dataset$safe_text = str_replace(new_dataset$safe_text, "<user>", " ")
new_dataset$safe_text = str_replace(new_dataset$safe_text, "<url>", " ")

In [105]:
head(new_dataset)

tweet_id,safe_text,label,sep
<fct>,<chr>,<dbl>,<dbl>
CL1KWCMY,Me &and; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St.,0,0
E3303EME,I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day,1,0
M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD",-1,0
1DR6ROZ4,"I mean if they immunize my kid with something that won't secretly kill him years down the line then I'm all for it, but I don't trust that",-1,0
J77ENIIE,Thanks to Catch me performing at La Nuit NYC 1134 1st ave. Show starts at 6! #jennifair #mmr…,0,0
OVNPOAUX,a nearly 67 year old study when mental health studies and vaccines were relatively in their infancies that has been refuted?,1,0


## Applying text pre-processing

In [106]:
# Cleaning the tweeter data from training set 
corpus = VCorpus(VectorSource(new_dataset$safe_text))
corpus = tm_map(corpus, content_transformer(tolower))
corpus = tm_map(corpus, removeNumbers)
corpus = tm_map(corpus, removePunctuation)
corpus = tm_map(corpus, removeWords, stopwords())
corpus = tm_map(corpus, stemDocument)
corpus = tm_map(corpus, stripWhitespace)


In [107]:
# Creating the Bag of Words model for train data
dtm = DocumentTermMatrix(corpus)
dtm = removeSparseTerms(dtm, 0.999)
df = as.data.frame(as.matrix(dtm))
df$label = new_dataset$label
df$sep = new_dataset$sep

In [108]:
# Sepataing the data back to training data and test data
library(dplyr)
dataset_train = df%>%
    filter(sep == 0)%>%
    select(-sep)

dataset_test = df%>%
    filter(sep == 1)%>%
    select(-sep)


In [ ]:
#write.csv(dataset_train, 'trainset.csv', row,names = FALSE)
#write.csv(dataset_test, 'testset.csv', row,names = FALSE)

In [109]:
head(dataset_train)

“user,abl,abort,absolut,abt,abus,access,accord,across,act,...,yesterday,yet,york,youll,young,your,yrs,zero,zombi,label
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# Splitting the dataset into the Training set and validation set
set.seed(123)
split = sample.split(dataset_train$label, SplitRatio = 0.8)
training_set = subset(dataset_train, split == TRUE)
validation_set = subset(dataset_train, split == FALSE)


## Modelling 

In [ ]:
# Fitting Random Forest Regression to the dataset

set.seed(1234)
regressor = randomForest(x = training_set[-1166],
                         y = training_set$label, .
                         ntree = 100)
# obtained RMSE = 0.598... on the Leaderboard

In [15]:
#Predicting new data using validation set
y_pred = predict(regressor, newdata = validation_set[-1166])
preds = data.frame(y_pred)

# Score
sqrt( mean( (preds$y_pred- validation_set$label)^2 , na.rm = TRUE ) ) # RMSE lower is better

[1] 0.5990679

In [ ]:
# Fitting xgboost model using caret package
control <- trainControl(method='cv', 
                        number=10)

regr = train(form = label ~ ., data = X_train, method = 'xgbTree')
regr
regr$bestTune

# RMSE = 0.574... on the leaderboard

In [ ]:
#Predicting new data using validation set
y_pred = predict(regressor, newdata = validation_set[-1166])
preds = data.frame(y_pred)

# Score
sqrt( mean( (preds$y_pred- validation_set$label)^2 , na.rm = TRUE ) ) # RMSE lower is better

In [59]:
# ONLY RUN THIS CODE IF YOU DID NOT GET ANY MISSING VALUES WHEN DOING EDA.
# ROWS ON SAMPLESUBMISSION FILE AND TEST DATA SHOULD BE IDENTICAL HENCE THIS CODE.
#head(ss)
#nrow(test)
tid = ss$tweet_id
tid[!tid %in% test$tweet_id] # Finding tweet ids not in test set - for the purposes of submission
tn = test$tweet_id
tn[!tn %in% ss$tweet_id]
ss = ss[-2453,] # Removing rows that have irregularities in the data
test = test[-2025,]


[1] H0VUUY2P
5177 Levels: 00BHHHP1 00UNMD0E 01AXPTJF 01HOEQJW 01JUKMAO 01V1X8XW ... ZZIYCVNH

[1] Dr. JAMES SHANNON
5177 Levels: 00BHHHP1 00UNMD0E 01AXPTJF 01HOEQJW 01JUKMAO 01V1X8XW ... ZZIYCVNH

In [16]:
#Predicting new data using test set
y_pred_test = predict(regressor, newdata = dataset_test[-1166])
preds = data.frame(y_pred_test)


In [17]:
# writing a csv of the predicted
ss$label = y_pred_test
head(ss)

tweet_id,label
<fct>,<dbl>
00BHHHP1,-0.02188889
00UNMD0E,0.73966667
01AXPTJF,0.04865900
01HOEQJW,0.41522222
01JUKMAO,-0.06633333
01V1X8XW,0.74911111


In [18]:
write.csv(ss, "Subfile.csv", row.names = FALSE)